In [1]:
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, BitsAndBytesConfig

local_model_path = r"D:\LLM_Demos\models\distilgpt2"

bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(local_model_path, quantization_config=bnb_config)

config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, bias="none")
lora_model = get_peft_model(model, config)

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=1,  # Very small batch size for low RAM
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs"
)

from datasets import load_dataset
dataset = load_dataset("tiny_shakespeare", trust_remote_code=True)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=dataset["train"],
)

trainer.train()

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [2]:
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

local_model_path = r"D:\LLM_Demos\models\distilgpt2"
model = AutoModelForCausalLM.from_pretrained(local_model_path)

config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, bias="none")
lora_model = get_peft_model(model, config)

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs"
)

from datasets import load_dataset
dataset = load_dataset("tiny_shakespeare", trust_remote_code=True)  

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=dataset["train"],
)

trainer.train()


C:\Users\Shubho\AppData\Roaming\Python\Python312\site-packages\peft\tuners\lora\layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [text]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [3]:
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Load local GPT-2 model
local_model_path = r"D:\LLM_Demos\models\distilgpt2"
model = AutoModelForCausalLM.from_pretrained(local_model_path)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Load dataset
dataset = load_dataset("tiny_shakespeare", trust_remote_code=True)

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# LoRA Config
config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, bias="none")
lora_model = get_peft_model(model, config)

# Training Config
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=1,  # Small batch size for low RAM
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    remove_unused_columns=False  # Prevents dropping important columns
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

trainer.train()


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [4]:
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Load local GPT-2 model
local_model_path = r"D:\LLM_Demos\models\distilgpt2"
model = AutoModelForCausalLM.from_pretrained(local_model_path)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Fix padding token issue
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding

# Update model embeddings if new tokens were added
model.resize_token_embeddings(len(tokenizer))

# Load dataset
dataset = load_dataset("tiny_shakespeare", trust_remote_code=True)

# Tokenize function
def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        truncation=True, 
        padding="max_length",  # Enables padding
        max_length=512  # Adjust based on dataset
    )

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# LoRA Config
config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, bias="none")
lora_model = get_peft_model(model, config)

# Training Config
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=1,  # Small batch size for low RAM
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    remove_unused_columns=False  # Prevents dropping tokenized columns
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

trainer.train()


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [5]:
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Load local GPT-2 model
local_model_path = r"D:\LLM_Demos\models\distilgpt2"
model = AutoModelForCausalLM.from_pretrained(local_model_path)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Fix padding token issue
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding

# Update model embeddings if new tokens were added
model.resize_token_embeddings(len(tokenizer))

# Load dataset
dataset = load_dataset("tiny_shakespeare", trust_remote_code=True)

# Tokenize function (Now includes `labels`)
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"], 
        truncation=True, 
        padding="max_length",  
        max_length=512  # Adjust based on dataset
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # Set labels for loss calculation
    return tokenized

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# LoRA Config
config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, bias="none")
lora_model = get_peft_model(model, config)

# Training Config
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=1,  # Small batch size for low RAM
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    remove_unused_columns=False  # Prevents dropping important columns
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

trainer.train()


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=4.274303436279297, metrics={'train_runtime': 3.8423, 'train_samples_per_second': 0.781, 'train_steps_per_second': 0.781, 'total_flos': 393304080384.0, 'train_loss': 4.274303436279297, 'epoch': 3.0})

In [1]:
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Load local GPT-2 model
local_model_path = r"D:\LLM_Demos\models\distilgpt2"
model = AutoModelForCausalLM.from_pretrained(local_model_path)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Fix padding token issue
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding

# Update model embeddings if new tokens were added
model.resize_token_embeddings(len(tokenizer))

# Load dataset
dataset = load_dataset("tiny_shakespeare", trust_remote_code=True)

# Tokenize function (Now includes `labels`)
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"], 
        truncation=True, 
        padding="max_length",  
        max_length=512  # Adjust based on dataset
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # Set labels for loss calculation
    return tokenized

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# LoRA Config
config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, bias="none")
lora_model = get_peft_model(model, config)

# Training Config
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=1,  # Small batch size for low RAM
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    remove_unused_columns=False  # Prevents dropping important columns
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

trainer.train()


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
C:\Users\Shubho\AppData\Roaming\Python\Python312\site-packages\peft\tuners\lora\layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=4.27440865834554, metrics={'train_runtime': 3.9689, 'train_samples_per_second': 0.756, 'train_steps_per_second': 0.756, 'total_flos': 393304080384.0, 'train_loss': 4.27440865834554, 'epoch': 3.0})

In [6]:
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
output = model.generate(**inputs, max_length=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time of the year, the city of San Francisco is a city of people. It is a city of people who are not just people who are not just people who are not just people who are not just people who are not just people who are not just people who are not just people who are not just because they are just people who are not just because they are just people who are not just because they are just because they are just because they are just because they are just because they


In [2]:
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Load local GPT-2 model
local_model_path = r"D:\LLM_Demos\models\distilgpt2"
model = AutoModelForCausalLM.from_pretrained(local_model_path)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Fix padding token issue
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding

# Update model embeddings if new tokens were added
model.resize_token_embeddings(len(tokenizer))

# Load dataset
dataset = load_dataset("tiny_shakespeare", trust_remote_code=True)

In [3]:
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Load local GPT-2 model
local_model_path = r"D:\LLM_Demos\models\distilgpt2"
model = AutoModelForCausalLM.from_pretrained(local_model_path)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Fix padding token issue
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding

# Update model embeddings if new tokens were added
model.resize_token_embeddings(len(tokenizer))

# Load dataset
dataset = load_dataset("tiny_shakespeare", trust_remote_code=True)
# Tokenize function (Now includes `labels`)
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"], 
        truncation=True, 
        padding="max_length",  
        max_length=512  # Adjust based on dataset
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # Set labels for loss calculation
    return tokenized

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# LoRA Config
config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, bias="none")
lora_model = get_peft_model(model, config)
# Training Config
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=1,  # Small batch size for low RAM
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    remove_unused_columns=False  # Prevents dropping important columns
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

trainer.train()

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

C:\Users\Shubho\AppData\Roaming\Python\Python312\site-packages\peft\tuners\lora\layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


TrainOutput(global_step=3, training_loss=4.274524688720703, metrics={'train_runtime': 3.6192, 'train_samples_per_second': 0.829, 'train_steps_per_second': 0.829, 'total_flos': 393304080384.0, 'train_loss': 4.274524688720703, 'epoch': 3.0})

In [4]:
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
output = model.generate(**inputs, max_length=100000)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


IndexError: index out of range in self

In [5]:
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
output = model.generate(**inputs, max_length=5000)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


IndexError: index out of range in self

In [6]:
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
output = model.generate(**inputs, max_length=1000)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time of the year, the most important thing to do is to be able to do it.



The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do is to do it.
The most important thing to do i